### Importing libraries

In [ ]:
library(funFEM)
library(tidyverse)
library(ggmap)
library(gridExtra)
library(RColorBrewer)
library(ggthemes)
data(velib)

### Preparing data

In [ ]:
x <- as.matrix(velib$data)
colnames(x) <- 1:ncol(x)
rownames(x) <- velib$names

bikes <- tibble(velib$data)
bikes$long <- velib$position$longitude
bikes$lat <- velib$position$latitude
bikes$hill <- as.factor(velib$bonus)
bikes$name <- velib$name

fig <- function(width, heigth){
     options(repr.plot.width = width, repr.plot.height = heigth)
}

name_func <- function(name) {
    paste('T', as.numeric(substring(name, first=2))-13, sep='')
}

bikes <- bikes %>% 
    select(!V1:V13) %>%
    rename_with(name_func, V14:V181)

head(bikes)

In [ ]:
x <- as.matrix(velib$data)
colnames(x) <- 1:ncol(x)
rownames(x) <- velib$names
n <- nrow(x)
stations <- 1:n 
dates <- 14:181
x <- x[stations, dates]
colnames(x) <- 1:length(dates)

### Descriptive statistics

In [ ]:
days = c('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')

bikes_long <- bikes %>% pivot_longer(cols=T1:T168, names_to='time', values_to ='loading') %>% 
    mutate(time=as.numeric(substring(time, first=2))-1)

bikes.load_by_day <- bikes_long %>% 
    mutate(day=factor(days[(time %/% 24) + 1], ordered = TRUE, levels=days)) %>%
    mutate(hour=time %% 24) %>%
    group_by(day, hour) %>%
    summarise(mean_load=mean(loading)) %>%
    mutate(usage=100*(1-mean_load))

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)
daycolors <- c(brewer.pal(name='YlGn', 6)[-1], brewer.pal(name='Blues', 9)[c(5,8)])
ggplot(bikes.load_by_day, aes(hour, usage, col=day)) + 
    geom_line() +
    geom_point() +
    scale_x_continuous(breaks = seq(0, 24, by=3)) +
    #scale_y_continuous(limits = c(0.5,0.75)) +
    scale_color_manual(labels=days, values=daycolors) +
    #labs(title='Average loading by time of day', x='hour', y='average loading', color='Day')
    labs(title='Average percentage of bikes in use by time and day', x='hour', y='usage', color='Day')

In [ ]:
bikes.load_by_hour <- bikes_long %>%
    filter(time < 5*24) %>%
    mutate(hour=time %% 24) %>%
    group_by(hour, name) %>%
    summarise(mean_load=mean(loading)) %>%
    inner_join(bikes %>% select(name, long, lat), by='name')

#bikes.load_by_hour <- bikes_long %>%
#    filter(time < 24) %>%
#    rename(mean_load = loading, hour=time)

options(repr.plot.width = 16, repr.plot.height = 5)

q1 <- qmplot(long, lat, data=bikes.load_by_hour %>% filter(hour==6), col=mean_load) +
    scale_colour_gradient(low='white', high='red', limits=c(0,1)) +
    labs(title='Average weekday loading at 6h') +
    guides(color='none')

q2 <- qmplot(long, lat, data=bikes.load_by_hour %>% filter(hour==12), col=mean_load) +
    scale_colour_gradient(low='white', high='red', limits=c(0,1)) +
    labs(title='Average weekday loading at 12h') +
    guides(color='none')

q3 <- qmplot(long, lat, data=bikes.load_by_hour %>% filter(hour==23), col=mean_load) +
    scale_colour_gradient(low='white', high='red', limits=c(0,1)) +
    labs(title='Average weekday loading at 23h') +
    guides(color='none')

grid.arrange(q1, q2, q3, ncol=3)

### Principal component analysis

In [ ]:
library(FactoMineR)
library(factoextra)

In [ ]:
num.pc <- 10
res.pca <- PCA(bikes %>% select(T1:T168), ncp=num.pc)

In [ ]:
fig(10,6)
fviz_screeplot(res.pca, ncp=num.pc, addlabels=TRUE) + 
    scale_x_discrete(breaks=c(1:num.pc), labels=paste('PC', c(1:num.pc), sep='')) +
    labs(x='', title='') +
    theme_gray()

In [ ]:
ggplot() + geom_bar(stat="identity",aes(x=c(1:num.pc), y=res.pca$eig[1:num.pc,3]), fill='#4984B3') +
    labs(x='', y='Percentage of variance', title='Cumulative percentage of variance') +
    scale_x_continuous(breaks = c(1:num.pc), labels=paste('PC', c(1:num.pc), sep='')) + 
    annotate('segment', x=0, xend=num.pc+1, y=80, yend=80, col='red', linetype=2, size=0.7) +
    coord_cartesian(xlim=c(0.5, num.pc+0.5))

In [ ]:
day.labs <- rep('', 15)
day.labs[2*c(1:7)] <- days
plot_pc <- function(pc, i, label.days=FALSE) {
    g <-ggplot() + geom_line(aes(c(0:167), pc[,i])) + labs(y='', title=sprintf('PC%d loading profile', i)) + 
        scale_y_continuous(name='', limits=c(-1, 1)) + 
        #scale_x_continuous(name='', limits=c(0, 168), breaks=12*c(0:14), labels=NULL) +
        coord_cartesian(xlim=c(7, 161), ylim=c(-1,1)) +
        annotate('rect', 
                 xmin = 24*c(0:6), xmax = 24*c(1:7), 
                 ymin = rep(-1,7), ymax = rep(1,7), 
                 fill = daycolors, alpha = .2)
    if (label.days) {
        g <- g + scale_x_continuous(name='', limits=c(0, 168), breaks=12*c(0:14), labels=day.labs)
    } else {
        g <- g + scale_x_continuous(name='', limits=c(0, 168), minor_breaks=6*c(0:28), breaks=NULL)
    }
    g
}

options(repr.plot.width = 12, repr.plot.height = 10)
p1 <- plot_pc(res.pca$var$coord, 1)
p2 <- plot_pc(res.pca$var$coord, 2)
p3 <- plot_pc(res.pca$var$coord, 3)
p4 <- plot_pc(res.pca$var$coord, 4)
p5 <- plot_pc(res.pca$var$coord, 5)
grid.arrange(p1,p2,p3,p4,p5, ncol=1)

In [ ]:
bikes.pca <- as_tibble(res.pca$ind$coord) %>%
    select(num_range('Dim.', 1:5)) %>%
    rename_at(vars(paste('Dim.', c(1:5), sep='')), ~ paste('PC', c(1:5), sep=''))

bikes.pca <- bind_cols(bikes %>% select(name, long, lat, hill), bikes.pca)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)

g <- ggplot(bikes.pca, aes(col=hill)) + 
    guides(color='none')

p1 <- g + geom_point(aes(PC1, PC2)) +
    labs(x='PC1', y='PC2')
p2 <- g + geom_point(aes(PC1, PC3)) +
    labs(x='', y='PC3')
p3 <- g + geom_point(aes(PC2, PC3)) +
    labs(x='PC2', y='')

lay <- rbind(c(2,3),
             c(1,NA))
grid.arrange(p1, p2, p3, layout_matrix = lay)

In [ ]:
color_grad <- scale_color_gradient2("Dim1",low = "blue", mid="white", high = "red", midpoint=0)

t <- c(1:168)
line.plot <- ggplot() + 
    scale_y_continuous(name='', labels=NULL, limits=c(-1, 1), breaks=c(-1,0,1), minor_breaks=seq(-1,1, by=.5))# +
    #theme(plot.margin=unit(c(0,0,0,0), 'in'))

p1 <- qmplot(long, lat, data=bikes.pca, col=PC1) +
    color_grad + guides(color='none') + labs(title='PC1') +
    theme(plot.margin=unit(c(0,0,0,0), 'in'))
l1 <- line.plot + geom_line(aes(t, res.pca$var$coord[,1]))

p2 <- qmplot(long, lat, data=bikes.pca, col=PC2) +
    color_grad + guides(color='none') + labs(title='PC2')

l2 <- line.plot + geom_line(aes(t, res.pca$var$coord[,2]))

p3 <- qmplot(long, lat, data=bikes.pca, col=PC3) +
    color_grad + guides(color='none') + labs(title='PC3')

l3 <- line.plot + geom_line(aes(t, res.pca$var$coord[,3]))

p4 <- qmplot(long, lat, data=bikes.pca, col=PC4) +
    color_grad + guides(color='none') + labs(title='PC4')

l4 <- line.plot + geom_line(aes(t, res.pca$var$coord[,4]))
    
#grid.arrange(p1, p2, p3, p4, nrow=2, ncol=2)
layout1 <- rbind(c(1,1,3,3,5,5),
                c(1,1,3,3,5,5),
                c(2,2,4,4,6,6))

layout2 <- rbind(c(1,1,3,3,5,5,7,7),
                c(1,1,3,3,5,5,7,7),
                c(2,2,4,4,6,6,8,8))

layout3 <- rbind(c(1,1,3,3),
                c(1,1,3,3),
                c(2,2,4,4),
                c(5,5,7,7),
                c(5,5,7,7),
                c(6,6,8,8))

options(repr.plot.width = 16, repr.plot.height = 4)
grid.arrange(p1, l1, p2, l2, p3, l3, p4, l4, layout_matrix=layout2)

### K-means clustering

In [ ]:
fig(10,5)
max_k <- 10
loss <- vector(length=max_k)
clust.data <- bikes.pca %>% select(PC1:PC5)
for (k in 2:max_k) {
    clust <- kmeans(clust.data, k)
    loss[k] = clust$tot.withinss
}
loss[1] = clust$totss
ggplot() + geom_bar(stat="identity",aes(x=c(1:max_k), y=loss), fill='#4984B3') +
    scale_x_continuous(breaks = c(1:max_k), name = 'Number of clusters') +
    scale_y_continuous(breaks = seq(0, loss[1], length.out=5), 
                       labels = seq(0, 1, length.out=5), 
                       name = 'Within-class variance')
    

In [ ]:
set.seed(423)
res.km5 <- kmeans(clust.data, 5, nstart=10)

fig(10,6)
#qmplot(long, lat, data=bikes.pca, col=as.factor(res.km5$cluster), source='stamen', maptype='toner', mapcolor='bw') +
#    labs(color='Cluster')

mapMargin <- 0.01
lowerleftlon = min(bikes$long) - mapMargin
lowerleftlat = min(bikes$lat) - mapMargin
upperrightlon = max(bikes$long) + mapMargin
upperrightlat = max(bikes$lat) + mapMargin
myMap <- get_stamenmap(bbox=c(lowerleftlon, lowerleftlat, upperrightlon, upperrightlat), 
                       zoom=13,maptype='toner', color='bw')
#?get_stamenmap
ggmap(myMap)

In [ ]:
ggmap(myMap) + geom_point(aes(long, lat), data=bikes, col=res.km5$cluster)

In [ ]:
K <- 5
p <- 5
pca.var_coord <- as.matrix(res.pca$var$coord[,1:5])
res.km5 <- kmeans(clust.data, 5, nstart=10)
km5centers <- matrix(nrow = 168, ncol = 5)
for (i in 1:5){
    km5centers[, i] <- res.pca$call$centre +
    res.pca$call$ecart.type * res.pca$var$coord[, 1:p] %*% (as.matrix(res.km5$centers[i, ], ncol = 1) / sqrt(res.pca$eig[1:p, 1])) # coord. in the orig. space
}

In [ ]:
day_lab <- rep('', 15)
day_lab[2*c(1:7)] <- days
plot_center <- function(centers, i) {
    ggplot() + geom_line(aes(t, centers[,i])) + labs(y='', title=sprintf('Centroid %d loading profile', i)) + 
        scale_y_continuous(name='', limits=c(0, 1)) + 
        scale_x_continuous(name='', limits=c(0, 168), breaks=12*c(0:14), labels=day_lab) +
        coord_cartesian(xlim=c(7, 161), ylim=c(0,1)) +
        annotate('rect', xmin = 24*c(0:6), xmax = 24*c(1:7), ymin = rep(0,7), ymax = rep(1,7), 
                 fill=daycolors, alpha = .2)
        
}

options(repr.plot.width = 12, repr.plot.height = 10)
p1 <- plot_center(km5centers, 1)
p2 <- plot_center(km5centers, 2)
p3 <- plot_center(km5centers, 3)
p4 <- plot_center(km5centers, 4)
p5 <- plot_center(km5centers, 5)
grid.arrange(p1,p2,p3,p4,p5, ncol=1)

In [ ]:
K <- 6
p <- 5
pca.var_coord <- as.matrix(res.pca$var$coord[,1:5])
res.km <- kmeans(clust.data, K, nstart=10)
kmcenters <- matrix(nrow = 168, ncol = K)
for (i in 1:K){
    kmcenters[, i] <- res.pca$call$centre +
    res.pca$call$ecart.type * res.pca$var$coord[, 1:p] %*% (as.matrix(res.km$centers[i, ], ncol = 1) / sqrt(res.pca$eig[1:p, 1])) # coord. in the orig. space
}

fig(12,10)
p1 <- plot_center(kmcenters, 1)
p2 <- plot_center(kmcenters, 2)
p3 <- plot_center(kmcenters, 3)
p4 <- plot_center(kmcenters, 4)
p5 <- plot_center(kmcenters, 5)
p6 <- plot_center(kmcenters, 6)
grid.arrange(p1,p2,p3,p4,p5,p6, ncol=1)

In [ ]:
set.seed(423)
km.full <- kmeans(bikes %>% select(T1:T168), 6, nstart=10)
km.full$centers <- km.full$centers[c(5,4,2,6,1,3),]
#km.full$cluster <- as.factor(km.full$cluster)
#levels(km.full$cluster) <- list('1'='5', '2'='4', '3'='2', '4'='6', '5'='1', '6'='3')
km.full$cluster <- factor(km.full$cluster, levels=c(1:6), labels=c('5','4','2','6','1','3'))

In [ ]:
fig(10,5)
qmplot(long, lat, data=bikes, col=as.factor(km.full$cluster)) + geom_point(size=3)

In [ ]:
compare_full_v_pca <- function(centers.full, centers.pca, i) {
    ggplot() + geom_line(aes(t, centers.full[i,])) + labs(y='') +
        geom_line(aes(t, centers.pca[,i]), color='red') + 
        scale_y_continuous(name='', limits=c(0, 1)) + 
        scale_x_continuous(name='', limits=c(0, 168), breaks=12*c(0:14), labels=day_lab) +
        coord_cartesian(xlim=c(7, 161), ylim=c(0,1)) +
        annotate('rect', xmin = 24*c(0:6), xmax = 24*c(1:7), ymin = rep(0,7), ymax = rep(1,7), 
                 fill=daycolors, alpha = .2)
        
}

fig(12,10)
p1 <- compare_full_v_pca(km.full$centers, kmcenters, 1)
p2 <- compare_full_v_pca(km.full$centers, kmcenters, 2)
p3 <- compare_full_v_pca(km.full$centers, kmcenters, 3)
p4 <- compare_full_v_pca(km.full$centers, kmcenters, 4)
p5 <- compare_full_v_pca(km.full$centers, kmcenters, 5)
p6 <- compare_full_v_pca(km.full$centers, kmcenters, 6)
grid.arrange(p1,p2,p3,p4,p5,p6, ncol=1)
#km.full$centers[1,]

In [ ]:
confusionMatrix(reference=as.factor(res.km$cluster), 
                data=as.factor(km.full$cluster), 
                positive=NULL, dnn=c('PCA clustering', 'Raw data clustering'))

In [ ]:
fig(12,12)

plotPCAclustering <- function(pca.data, clusters, ...) {
    g <- ggplot(pca.data, aes(col=as.factor(clusters))) +
        guides(color='none') +
        scale_x_continuous(position='top')

    p12 <- g + geom_point(aes(PC1, PC2), ...) +
        labs(x='PC1', y='PC2')
    p13 <- g + geom_point(aes(PC1, PC3), ...) +
        labs(x='', y='PC3')
    p14 <- g + geom_point(aes(PC1, PC4), ...) +
        labs(x='', y='PC4')
    p23 <- g + geom_point(aes(PC2, PC3), ...) +
        labs(x='PC2', y='')
    p24 <- g + geom_point(aes(PC2, PC4), ...) +
        labs(x='', y='')
    p34 <- g + geom_point(aes(PC3, PC4), ...) +
        labs(x='PC3', y='')
    
    lay <- rbind(c(1,NA,NA),
                 c(2, 4,NA),
                 c(3, 5, 6))
    grid.arrange(p12, p13, p14, p23, p24, p34, layout_matrix = lay)
}

In [ ]:
fig(10,10)
plotPCAclustering(bikes.pca, res.km$cluster)

### Herarchical clustering

In [ ]:
D <- dist(bikes.pca %>% select(PC1:PC5))
bikes.hc <- hclust(D, 'ward.D2')

In [ ]:
hc.clusters <- cutree(bikes.hc, 4)
plotPCAclustering(bikes.pca, hc.clusters)

### Gaussian mixture model

In [ ]:
library(mclust)

In [ ]:
mod.gm = Mclust(bikes.pca %>% select(PC1:PC5), G=6, model='VVV')
summary(mod.gm)

In [ ]:
fig(10,10)
plotPCAclustering(bikes.pca, mod.gm$classification)

In [ ]:
mod.gm4 = Mclust(bikes.pca %>% select(PC1:PC5), G=4, model='VVV')
plot(mod.gm4, what='density')

In [ ]:
K <- 4
p <- 5
pca.var_coord <- as.matrix(res.pca$var$coord[,1:5])
res.gmm <- Mclust(bikes.pca %>% select(PC1:PC5), G=K, model='VVV')
centers <- matrix(nrow = 168, ncol = K)
for (i in 1:K){
    centers[, i] <- res.pca$call$centre +
    res.pca$call$ecart.type * res.pca$var$coord[, 1:p] %*% (as.matrix(res.gmm$parameters$mean[,i], ncol = 1) / sqrt(res.pca$eig[1:p, 1])) # coord. in the orig. space
}

options(repr.plot.width = 12, repr.plot.height = 10)
p1 <- plot_center(centers, 1)
p2 <- plot_center(centers, 2)
p3 <- plot_center(centers, 3)
p4 <- plot_center(centers, 4)
#p5 <- plot_center(centers, 5)
#p6 <- plot_center(centers, 6)
grid.arrange(p1,p2,p3,p4, ncol=1)